<a href="https://colab.research.google.com/github/huydoan98tl/AI_Project/blob/main/training_gender_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is file to train model to predict gender. AI_SEM221


## I. Prepare

In [ ]:
#link drive
from google.colab import drive
drive.mount('/content.drive/')

In [ ]:
# change to current folder
%cd /content/drive/MyDrive/Gender_recognition
! ls

/content/drive/MyDrive/Gender_recognition
gender-classification-dataset.zip  training_gender_recognition.ipynb
kaggle.json			   Validation
Training


# II. dowload dataset from kaggle

In [ ]:
from google.colab import files
!pip install -q kaggle

In [ ]:
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

In [ ]:
!kaggle datasets download -d cashutosh/gender-classification-dataset

 99% 267M/269M [00:10<00:00, 27.2MB/s]
100% 269M/269M [00:11<00:00, 25.7MB/s]


In [ ]:
!ls

gender-classification-dataset.zip  training_gender_recognition.ipynb
kaggle.json


In [ ]:
!unzip gender-classification-dataset.zip

# III. Import lib

In [ ]:
import os, cv2, random
import numpy as np
import tensorflow as tf
import glob
from tensorflow import keras
from keras_preprocessing.image import img_to_array
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import load_model
from keras import layers  
from keras.layers import BatchNormalization, Conv2D, MaxPool2D, Activation, Flatten, Dropout, Dense
from sklearn.model_selection import train_test_split

# IV.Main

## 4.1 initial parameter

In [3]:
# output of cascade model is gray images with size 96x96
WIDTH = 96
HEIGHT = 96
#CHANNEL = 3

# define
class_number = 2
batch_size = 128
epochs = 50
path_train = '/content/drive/MyDrive/Gender_recognition/Training'
path_test = '/content/drive/MyDrive/Gender_recognition/Validation'

# initial list of input image
data_train = []
label_train = []
data_test = []
label_test = []
count_train = 0
count_test = 0

# get all images in traning folder
img_trainlist = [img for img in glob.glob(path_train + '/**/*', recursive = True) if not os.path.isdir(img)]
random.shuffle(img_trainlist)

# get all images in validation folder
img_testlist = [img for img in glob.glob(path_test + '/**/*', recursive = True) if not os.path.isdir(img)]
random.shuffle(img_testlist)


## 4.2. Load image

In [ ]:
# add data and label of train
for img_train in img_trainlist:
  #read image
  img_read = cv2.imread(img_train)
  img_read = cv2.resize(img_read,(WIDTH, HEIGHT))
  img_read = img_to_array(img_read)
  
  #add data
  data_train.append(img_read)
  
  #split path to take the name of folder
  label_value = img_train.split(os.path.sep)[-2] 
  if label_value == 'female':
    label_value = 1
  else:
    label_value = 0
    label_train.append(label_value) # add label

  # print out number of image training
  count_train = count_train + 1
  print("Processing Training Images: ", count_train)

# add data and label of test
for img_test in img_testlist:
  #read image
  img_read = cv2.imread(img_test)
  img_read = cv2.resize(img_read,(WIDTH, HEIGHT))
  img_read = img_to_array(img_read)
  
  #add data
  data_test.append(img_read)
  
  #split path to take the name of folder
  label_value = img_test.split(os.path.sep)[-2] 
  if label_value == 'female':
    label_value = 1
  else:
    label_value = 0
  label_test.append(label_value) #add label

  # print out number of image testing
  count_test = count_test + 1
  print("Processing Testing Image: ", count_test)

#conver list to array
data_train = np.array(data_train, dtype = 'float')/255.0
label_train = np.array(label_train)
data_test = np.array(data_test, dtype = 'float')/255.0
label_test = np.array(label_test)

# covert to binary matrix
label_train = keras.utils.to_categorical(label_train, class_number)
label_test = keras.utils.to_categorical(label_test, class_number)

## 4.3 Build Model